In [34]:
import os

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from PIL import Image


### Util Functions

In [10]:
CONFIG = {
    "seed" : 42,
    "img_size" : 512,
    "num_class": 5,
    "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    # model cfg
    "backbone": "tf_efficientnetv2_s_in21ft1k",
}


In [4]:
def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed(CONFIG["seed"])


In [15]:
DATA_DIR = "./data/"
TEST_DIR = './data/test_thumbnails'
ALT_TEST_DIR = './data/test_images'
BEST_WEIGHT = "./output/best.pth"


### Dataset defination

In [29]:
from pathlib import Path
from torchvision import transforms

class UBCOCEANDataset(Dataset):
    def __init__(self, root: str, infocsv: pd.DataFrame, num_class: int = 5) -> None:
        self.data_dir = Path(root, "test_thumbnails")
        self.info = infocsv
        self.label_encoder = {
            "HGSC": 0,
            "EC": 1,
            "CC": 2,
            "LGSC": 3,
            "MC": 4
        }
        self.transform = transforms.Compose([
            transforms.Resize(size=(512, 512)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    
    def __len__(self) -> int:
        return len(self.info)
    
    def __getitem__(self, index) -> torch.Tensor:
        datapiece = self.info.iloc[index]
        image_id = datapiece["image_id"]
        image = Image.open(Path(self.data_dir, f"{datapiece['image_id']}_thumbnail.png"))
        image = self.transform(image)
        return image_id, image

test_info = pd.read_csv(Path(DATA_DIR, "test.csv"))
Test_thumbnail_dataset = UBCOCEANDataset(root=DATA_DIR, infocsv=test_info, num_class=CONFIG["num_class"])
test_dataloader = DataLoader(dataset=Test_thumbnail_dataset, shuffle=False, batch_size=1, num_workers=2, pin_memory=True)


### Model defination

In [24]:
import timm

class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super(GeM, self).__init__()
        self.p = nn.Parameter(torch.ones(1)*p)
        self.eps = eps

    def forward(self, x):
        return self.gem(x, p=self.p, eps=self.eps)
        
    def gem(self, x, p=3, eps=1e-6):
        return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)
        
    def __repr__(self):
        return self.__class__.__name__ + \
                '(' + 'p=' + '{:.4f}'.format(self.p.data.tolist()[0]) + \
                ', ' + 'eps=' + str(self.eps) + ')'


class UBCModel(nn.Module):
    def __init__(self, cfg):
        super(UBCModel, self).__init__()
        self.model = timm.create_model(cfg["backbone"], pretrained=True)
        for param in self.model.parameters():
            param.requires_grad = False

        in_features = self.model.classifier.in_features
        self.model.classifier = nn.Identity()
        self.model.global_pool = nn.Identity()
        self.pooling = GeM()
        self.linear = nn.Linear(in_features, cfg["num_class"])
        self.softmax = nn.Softmax(dim=1)

    def forward(self, images):
        features = self.model(images)
        pooled_features = self.pooling(features).flatten(1)
        output = self.linear(pooled_features)
        return output
    
    def get_pred(self, logit):
        return torch.max(self.softmax(logit), 1)[1]


In [25]:
model = UBCModel(CONFIG)
checkpoint = torch.load(BEST_WEIGHT)
model.load_state_dict(checkpoint["model_ckp"])
model.to(CONFIG["device"])
model.eval()


UBCModel(
  (model): EfficientNet(
    (conv_stem): Conv2dSame(3, 24, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (act1): SiLU(inplace=True)
    (blocks): Sequential(
      (0): Sequential(
        (0): ConvBnAct(
          (conv): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (act1): SiLU(inplace=True)
        )
        (1): ConvBnAct(
          (conv): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (act1): SiLU(inplace=True)
        )
      )
      (1): Sequential(
        (0): EdgeResidual(
          (conv_exp): Conv2dSame(24, 96, kernel_size=(3, 3), stride=(2, 2), bias=False)
          (bn1): BatchNorm2d(96, eps=0

### Testing

In [31]:
from tqdm import tqdm
label_encoder = {
        0: "HGSC",
        1: "EC",
        2: "CC",
        3: "LGSC",
        4: "MC",
    }

sub_info = pd.read_csv(Path(DATA_DIR, "sample_submission.csv"))

predictions = []

for image_id, img in tqdm(test_dataloader):
    img = img.to(CONFIG["device"])
    print(img.shape)
    output = model(img)
    pred = model.get_pred(output).cpu().detach().item()
    predictions.append(label_encoder[pred])

sub_info["label"] = np.array(predictions)


100%|██████████| 1/1 [00:00<00:00,  4.01it/s]

torch.Size([1, 3, 512, 512])


In [32]:
sub_info.to_csv("submission.csv", index=False)


In [33]:
sub_info


,image_id,label
0,41,HGSC
